In [13]:
import numpy as np
import pandas as pd
from simple.renko import Renko
from simple.plotly import chartFigure
from simple.funcs import symlog

In [41]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
T

rec.array([('2022-05-08T00:00:00.048000', 2634.82,   31.), ('2022-05-08T00:00:00.090000', 2634.82,   10.), ('2022-05-08T00:00:06.184000', 2634.81,  -59.),
           ('2022-05-08T00:00:06.585000', 2634.82,   16.), ('2022-05-08T00:00:07.116000', 2634.82,   11.), ..., ('2022-05-08T00:59:59.571000', 2617.99,   -2.),
           ('2022-05-08T00:59:59.584000', 2617.99,   -2.), ('2022-05-08T00:59:59.596000', 2617.99,   -2.), ('2022-05-08T00:59:59.706000', 2618.  ,   32.),
           ('2022-05-08T00:59:59.826000', 2618.  , 1912.)],
          dtype={'names':['DateTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<f8','<f8'], 'offsets':[0,16,24], 'itemsize':40})

In [42]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]
Buy

rec.array([('2022-05-08T00:00:00.048000', 2634.82,   31.), ('2022-05-08T00:00:00.090000', 2634.82,   10.), ('2022-05-08T00:00:06.585000', 2634.82,   16.),
           ('2022-05-08T00:00:07.116000', 2634.82,   11.), ('2022-05-08T00:00:12.079000', 2634.82,   41.), ..., ('2022-05-08T00:59:58.970999', 2618.02,   40.),
           ('2022-05-08T00:59:58.977000', 2618.02,  250.), ('2022-05-08T00:59:59.488000', 2618.02,    9.), ('2022-05-08T00:59:59.706000', 2618.  ,   32.),
           ('2022-05-08T00:59:59.826000', 2618.  , 1912.)],
          dtype={'names':['DateTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<f8','<f8'], 'offsets':[0,16,24], 'itemsize':40})

In [43]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S

rec.array([('2022-05-08T00:00:00.055000', 2634.82, 2634.81), ('2022-05-08T00:00:00.088000', 2634.82, 2634.81),
           ('2022-05-08T00:00:00.119999', 2634.82, 2634.81), ('2022-05-08T00:00:01.243000', 2634.82, 2634.81),
           ('2022-05-08T00:00:03.632000', 2634.82, 2634.81), ..., ('2022-05-08T00:59:59.858000', 2618.  , 2617.97),
           ('2022-05-08T00:59:59.891000', 2618.  , 2617.97), ('2022-05-08T00:59:59.923000', 2618.  , 2617.97),
           ('2022-05-08T00:59:59.955000', 2618.  , 2617.97), ('2022-05-08T00:59:59.989000', 2618.  , 2617.97)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [44]:
len(T), len(S)

(98555, 101605)

In [45]:
chartFigure(Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3),
            Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3),
            Ask=dict(x=S.DT, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
            Bid=dict(x=S.DT, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines')
)

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'size': array([8.077…

In [50]:
step = 1
rnk = Renko(step, T.PriceA)
rnk.create_renko()

X = pd.DataFrame(rnk.bricks)
X['mid'] = (X.close + X.open) / 2
renko = X.mid
renko

0      2634.82
1      2634.32
2      2635.32
3      2636.32
4      2637.32
        ...   
100    2619.32
101    2618.32
102    2617.32
103    2616.32
104    2617.32
Name: mid, Length: 105, dtype: float64

In [51]:
X

,type,open,close,low,high,mid
0,first,2634.82,2634.82,NaN,NaN,2634.82
1,down,2634.82,2633.82,NaN,NaN,2634.32
2,up,2634.82,2635.82,2633.30,NaN,2635.32
3,up,2635.82,2636.82,2634.54,NaN,2636.32
4,up,2636.82,2637.82,NaN,NaN,2637.32
...,...,...,...,...,...,...
100,down,2619.82,2618.82,NaN,2621.04,2619.32
101,down,2618.82,2617.82,NaN,2619.85,2618.32
102,down,2617.82,2616.82,NaN,2619.68,2617.32
103,down,2616.82,2615.82,NaN,2618.10,2616.32
